# P4 Assignment

For this assignment, you will be writing P4 code which enables a switch to function as a joint router and firewall for an IPv6 network. The network topology is shown below.
<img src="./images/assignment4_topology.png" width="1000px">

Note: h23 will only be configured for a final test case, and begins unconfigured.

Authored by Alexander Wolosewicz, IIT, adapting some code by Nishanth Shyamkumar, IIT


## Create topology
The creation of the topology is done automatically for you. If you ever close the Jupyter kernel, or it crashes, all variable values are lost. The below cell can be executed to reset all the variable values.
Note that if the slice expires (24 hours after creation) you must start from the beginning. Any progress made in files (the shell script or P4 code) is safe as long as it is saved.
Make sure to retype your student ID for "student_id".

In [1]:
student_id = "btursunbayev"
verbose = False

In [2]:
# Setup slice, or retrieve values if slice already exists
from scripts import funcs

slice_values = funcs.setup(student_id=student_id, verbose=verbose)

vslice = slice_values['slice']
configs = slice_values['configs']
ips = slice_values['ips']
macs = slice_values['macs']
ifaces = slice_values['ifaces']
subnets = slice_values['subnets']
hosts = slice_values['hosts']
switches = slice_values['switches']
gateways = slice_values['gateways']

ModuleNotFoundError: No module named 'fabrictestbed_extensions'

In [ ]:
# If the above had any node fail, UNCOMMENT the line below and execute this cell, then try again
#slice.delete()

# P4 Programming
The main objective of this assignment is to create P4 code which will configure the switches (s1 and s2 in the topology) to function as IPv6 routers and firewalls. A copy of the P4 program from the tutorial, which created an IPv4 router, is provided as a starter template in [assignment.p4](./assignment.p4).

The routing requirements are that:
- hosts on each subnet can ping each other
- hosts can ping hosts on the other subnet, and the switches route to the other subnet using LPM

The firewall requirement is that TCP or UDP packets must have one of the ports listed below as either the source or destination port.

Besides modifying P4 code, you will also need to edit [add_rules_s1.sh](./add_rules_s1.sh) and [add_rules_s2.sh](./add_rules_s2.sh) to contain the control plane commands the switches need to be configured with.

In a Jupyter cell, `funcs.start_switches(switches, verbose)` and `funcs.stop_switches(switches, verbose)` will respectively start and stop the switches (see the start and end of the Testing section).

In [ ]:
# For convenience, list all the values from the config file
funcs.print_values(configs, macs, ips)

# SSH Keys
The below cell will print the ssh keys, which you can enter into a new terminal tab to ssh to that client. Useful for testing/debugging your P4 code by:
<list>
    <li> SSHing into two terminals for s1 - run ./run_bm2_s1.sh in s1, let it start, then ./add_rules_s1.sh, and the same for s2 </li>
    <li> SSHing into a host and attempting to ping another with ping6, or using send.py to send a packet with specific TCP/UDP ports </li>
</list>

In [ ]:
for node in vslice.get_nodes():
    print(f"{node.get_name()}: {node.get_ssh_command()}")

# Testing
The below cells will automatically test your P4 and sh scripts. The score breakdown for this assignment is:
- 2.5pts for h11 through h21 being able to ping each other
- 5 pts for the switches correctly filtering TCP and UDP ports
- 2.5pts for correctly routing to a random host added to subnet 2

There is a total of 10 points. These cells should not be modified, and they should be run in order.

In [ ]:
# Start switches before the tests - this also uploads your most recent P4 and add_rules files
funcs.start_switches(switches, verbose)

In [ ]:
# Ping Test
pingscore = funcs.ping_test(hosts, ips)

In [ ]:
# Port Test
portscore = funcs.port_test(hosts, ips, configs, ifaces, student_id, verbose)

In [ ]:
# h23 Test
h23score = funcs.h23_test(hosts, ips, switches, ifaces, configs, macs, gateways, verbose)

print(f"Total Score: {pingscore+portscore+h23score}/10pts")

In [ ]:
# To stop the switches, so they can be restarted (such as to update their P4 code)
funcs.stop_switches(switches, verbose)

In [ ]:
# Run this to generate the .tar.gz file you will submit to Canvas
funcs.create_tar(student_id, hosts, switches, override=True)

In [ ]:
# Uncomment and delete your slice when you're finished
# vslice.delete()